In [1]:
import torch
from IPython import display
from d2l import torch as d2l

In [2]:
batch_size = 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size)

C:\Users\25832\Anaconda3\lib\site-packages\torchvision\datasets\mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [3]:
next(iter(train_iter))

[tensor([[[[0.0000, 0.0039, 0.0078,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0078, 0.0039,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0000,  ..., 0.0118, 0.0000, 0.0000],
           ...,
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]]],
 
 
         [[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           ...,
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]]],
 
 
         [[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000

In [4]:
next(iter(test_iter))[0].shape

torch.Size([256, 1, 28, 28])

In [5]:
num_inputs = 784  #(28*28)
num_ouputs = 10   #(class)

In [6]:
w = torch.normal(0,0.001,size=(num_inputs,num_ouputs),requires_grad=True)

In [7]:
b = torch.zeros(num_ouputs,requires_grad=True)

In [8]:
def softmax(X):
    X_exp = torch.exp(X)
    partition = X_exp.sum(1,keepdim=True)
    return X_exp/partition

In [9]:
X = torch.normal(0,1,(2,5))
X

tensor([[-0.4269, -1.3697, -0.4633, -0.0968, -0.8891],
        [ 0.2788,  0.4036,  0.9790, -1.2508, -0.1339]])

In [11]:
X_prob = softmax(X)
X_prob

tensor([[0.2286, 0.0890, 0.2204, 0.3180, 0.1440],
        [0.1990, 0.2254, 0.4008, 0.0431, 0.1317]])

In [12]:
X_prob.sum(1,keepdim=True)

tensor([[1.0000],
        [1.0000]])

In [14]:
def net(X):
    return torch.matmul(X.reshape(-1,w.shape[0]),w) + b

In [22]:
y = torch.tensor([0, 2])
y_hat = torch.tensor([[0.1, 0.3, 0.6], [0.3, 0.2, 0.5]])
y_hat[[0, 1], y]

tensor([0.1000, 0.5000])

In [23]:
def cross_entropy(y_hat,y):
    return -torch.log(y_hat[range(len(y_hat)),y])
cross_entropy(y_hat,y)

tensor([2.3026, 0.6931])

In [24]:
def accuracy(y_hat,y):
    if len(y_hat.shape)>1 and y_hat.shape[1]>1:
        y_hat = y_hat.argmax(axis=1)
    cmp = y_hat.type(y.dtype)==y
    return float(cmp.type(y.dtype).sum())

In [26]:
accuracy(y_hat,y)/len(y)

0.5